# Prediction of the number of instructions that will be executed by OpenCL kernels given the global NDRange, via regression on the experimental data gathered by the oclude profiling tool

In this notebook, we are going to create machine learning models based on various regression techniques that will attempt to predict the total number of instructions per type that would be executed by an OpenCL kernel, given a specific global NDRange.

The data we will use come from the experiments that were conducted on a number of OpenCL kernels from the [rodinia benchmark suite](https://github.com/yuhc/gpu-rodinia), with the help of the [oclude profiling tool](https://github.com/zehanort/oclude).

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#0.-Libraries-and-helper-functions" data-toc-modified-id="0.-Libraries-and-helper-functions-1">0. Libraries and helper functions</a></span></li><li><span><a href="#1.-Introduction" data-toc-modified-id="1.-Introduction-2">1. Introduction</a></span></li><li><span><a href="#2.-Regression-models-used" data-toc-modified-id="2.-Regression-models-used-3">2. Regression models used</a></span><ul class="toc-item"><li><span><a href="#2.1-Linear-regression" data-toc-modified-id="2.1-Linear-regression-3.1">2.1 Linear regression</a></span></li><li><span><a href="#2.2-Elastic-Net-regression" data-toc-modified-id="2.2-Elastic-Net-regression-3.2">2.2 Elastic Net regression</a></span></li><li><span><a href="#2.3-Polynomial-regression" data-toc-modified-id="2.3-Polynomial-regression-3.3">2.3 Polynomial regression</a></span><ul class="toc-item"><li><span><a href="#2.3.1-Linear-based" data-toc-modified-id="2.3.1-Linear-based-3.3.1">2.3.1 Linear based</a></span></li><li><span><a href="#2.3.2-Elastic-Net-based" data-toc-modified-id="2.3.2-Elastic-Net-based-3.3.2">2.3.2 Elastic Net based</a></span></li></ul></li></ul></li><li><span><a href="#3.-Selecting-a-regression-model-per-kernel" data-toc-modified-id="3.-Selecting-a-regression-model-per-kernel-4">3. Selecting a regression model per kernel</a></span><ul class="toc-item"><li><span><a href="#3.1-Hyperparameter-optimization-across-multiple-models" data-toc-modified-id="3.1-Hyperparameter-optimization-across-multiple-models-4.1">3.1 Hyperparameter optimization across multiple models</a></span></li><li><span><a href="#3.2-Final-model-selection" data-toc-modified-id="3.2-Final-model-selection-4.2">3.2 Final model selection</a></span></li><li><span><a href="#3.3-Model-selection-results" data-toc-modified-id="3.3-Model-selection-results-4.3">3.3 Model selection results</a></span></li></ul></li></ul></div>

## 0. Libraries and helper functions

In [1]:
%%capture
!pip install --upgrade numpy
!pip install --upgrade scipy
!pip install --upgrade scikit-learn
!pip install --upgrade pandas
!pip install --upgrade plotly
!pip install --upgrade matplotlib
!pip install --upgrade seaborn

In [2]:
### basic imports ###

import os
import itertools
from collections import defaultdict
from time import time
import numpy as np
from scipy.stats.mstats import gmean
import pandas as pd
import seaborn as sb
import plotly.graph_objects as go
import plotly.offline as pyo
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
from IPython.display import display, HTML
from pprint import pprint

# for plotly: set notebook mode to work in offline
pyo.init_notebook_mode(connected=True)

### regression models and related stuff ###
from sklearn.linear_model import LinearRegression, MultiTaskElasticNet
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error

### globals go here ###
outdir = os.path.join(os.pardir, 'desk', 'outputs')

%matplotlib inline

In [3]:
# some helper functions, do not bother with them

def sorted_filenames_keyfunc(filename):
     return int(filename.split('__')[3].split('.')[0])

def get_grouped_all_profilings_filenames():
    
    def groupby_keyfunc(filename):
        return filename.split('__')[:3]

    sorted_filenames = sorted(os.listdir(outdir), key=groupby_keyfunc)

    retlist = []
    for k, v in itertools.groupby(sorted_filenames, groupby_keyfunc):
        retlist.append((k, sorted(v, key=sorted_filenames_keyfunc)))
    return retlist

def get_gsize_from_profiling_filename(filename):
    return int(filename.split('__')[-1].split('.')[0])

def get_average_instcounts_dicts(arg, discard_zeros=False):
    if isinstance(arg, list) or isinstance(arg, tuple):
        return list(map(get_average_instcounts_dicts, arg))

    from collections import Counter
    from functools import reduce
    from operator import add

    if not isinstance(arg, dict):
        arg = load_profiling_dicts(arg)

    results = arg['results']
    samples = len(results)

    if discard_zeros:
        avg_instcounts = dict(
            reduce(add, map(Counter, map(lambda x : x['instcounts'], results)))
        )
    else:
        c = Counter()
        for d in map(lambda x : x['instcounts'], results):
            c.update(d)
        avg_instcounts = dict(c)

    avg_instcounts = {
        k : int(v) // samples for k, v in avg_instcounts.items()
    }

    if 'timeit' in results[0]:
        avg_timeit = dict(
            reduce(add, map(Counter, map(lambda x : x['timeit'], results)))
        )
        avg_timeit = {
            k : v / samples for k, v in avg_timeit.items()
        }
    else:
        avg_timeit = None

    avg_dict = {}
    for k in filter(lambda x : x != 'results', arg.keys()):
        avg_dict[k] = arg[k]

    avg_dict['results'] = dict(instcounts=avg_instcounts)
    if avg_timeit:
        avg_dict['results']['timeit'] = avg_timeit

    return avg_dict

def get_list_of_profilings_filenames(benchmark=None, file=None, kernel=None):
    
    if benchmark is None:
        return os.listdir(outdir)

    profilings_filenames = list(filter(
        lambda x : x.startswith('__'.join(benchmark, file, kernel) + '__'),
        os.listdir(outdir)
    ))
    
    return list(map(lambda x : os.path.join(outdir, x), profilings_filenames))

def get_sorted_list_of_profilings_filenames(benchmark=None, file=None, kernel=None):
    return sorted(
        get_list_of_profilings_filenames(benchmark, file, kernel),
        key=sorted_filenames_keyfunc
    )

def load_profiling_dicts(arg, append_gsize=True):
    if isinstance(arg, list) or isinstance(arg, tuple):
        return list(map(load_profiling_dicts, arg))

    from json import load
    with open(os.path.join(outdir, arg), 'r') as f:
        profiling_dict = load(f)

    if append_gsize:
        profiling_dict['gsize'] = get_gsize_from_profiling_filename(arg)

    return profiling_dict

def remove_zeros(df):
    return df.loc[:, (df != 0).any(axis=0)]

## 1. Introduction

In this notebook, we will design a strategy to build a different regression model per kernel in order to fit the function $f_{kernel}$ as best as possible, where:

$$instcounts = f_{kernel}(gsize)$$

Obviously, the $f_{kernel}$ function is kernel-specific, because instruction counts are related to the $gsize$ input parameter in different ways depending on what each kernel is designed to do (i.e., depending on its *source code*).

Given our prior knowledge from the EDA we conducted (see EDA notebook), we assume we are not going to need regressors of order greater than that of the polynomial with order 2. More specifically, we assume that:
- The $f_{kernel}$ function of the "relatively fast" kernels can be satisfactorily approached with a linear regression model.
- The $f_{kernel}$ function of the "relatively slow" kernels can be satisfactorily approached with a polynomial regression model of order no greater than 2.

These 2 hypotheses are derived from the graphs we saw in the EDA notebook.

Furthermore, it should be noted that more sophisticated tree regression algorithms, like gradient boosting or even XGBoost, can not be used in our case. The reason is that no tree regressor is good at *extrapolating*, i.e. predicting values outside of the feature domain on which they were trained. This is crucial for our application, given that we need our regressors to predict instcounts for values of $gsize$ that they have never seen, and possibly orders of magnitude larger.

Therefore, we are going to use 4 different regression models, 2 linear and 2 polynomial, where the latter will be built on the former respectively.

## 2. Regression models used

In this section, we will briefly discuss the regression models that we used. More specifically, we will refer to:

- What each model does,
- On what type of data each model is expected to perform better/worse,
- Why we decided to use it, and
- What is expected of it regarding its performance.

### 2.1 Linear regression

The **linear regression model** fits an equation of the following form to the input data:

$$y = b_0 + b_1x_1 + b_2x_2 + ... + b_nx_n$$

where $y$ is the independent variable, $x_i$ are the dependent variables or features, $b_i$ are the coefficients the model tries to estimate and $n$ is the number of features. The goal of the $b_i$ estimation algorithm is to minimize the *mean squared error* between the calculated line and the actual data.

In our case, we have a *multioutput regression*, i.e. we have a vector $\vec{y}$ instead of a scalar $y$ as our independent variable. More specifically, $\vec{y}$ holds the counts of the executed instructions (aka "instcounts"), and the length of $\vec{y}$ equals to the number of the instructions of the LLVM bitcode instruction set which, at the time of the writing, equals about $70$. Moreover, we have *a single independent variable, the $gsize$ parameter*.

Therefore, in our case, the equation we are trying to fit looks more like that:

$$
\vec{y} = \vec{b}x \Leftrightarrow
\begin{bmatrix}
    counter_{add} \\
    counter_{sub} \\
    counter_{mul} \\
    \vdots
\end{bmatrix} =
\begin{bmatrix}
    coef_{add} \\
    coef_{sub} \\
    coef_{mul} \\
    \vdots
\end{bmatrix} gsize
$$

and what we try to estimate is \vec{b}, the vector of the coefficients.

A last important note on linear regression is that it is **strongly biased**, given that it assumes that the dependent variable is a linear function of the dependent variables (i.e. the features). Therefore, it seems a more suitable choice for the "relatively fast" kernels.

### 2.2 Elastic Net regression

**Elastic Net** is a linear regression algorithm that uses *regularization* techniques in order to solve an ill-posed problem or to prevent overfitting (it should be noted that we are most interested in the former in our case). Elastic net attempts to minimize the same objective function as that of the simple linear regression (mean squared error), plus the $l1$ and $l2$ norms (namely Manhattan and Euclidean, respectively) of the coefficients vector:

$$
    error_{elastic\ net} = error_{linear\ regression}
    + \lambda_1 \lVert \vec{b} \rVert
    + \lambda_2 \sqrt{ \lVert \vec{b} \rVert ^2 }
$$

$\lambda_1$ and $\lambda_2$ are weights that regulate which norm to take more into account, and are the most important hyperparameters of the Elastic Net model. It should be noted that the actual equation is more complex and contains some more hyperparameters, but the above form is sufficient for the supervisory purpose of this section.

By taking both these norms into consideration, Elastic Net attempts to shrink (or even eliminate) the contribution of certain features to the dependent variable and does a better job when dealing with correlated features, which is a plus in our case.

Further analysis regarding the algorithmic and mathematical properties of Elastic Net is considered to be outside the scope of this dissertation.

### 2.3 Polynomial regression

**Polynomial regression** tries to fit a polynomial to the training data instead of a line. It does so in 2 steps:

1. Computes the **polynomial features** of the training set, i.e. all the products and the powers of the features of the training set samples, up to a certain order. For example, if one of the samples is the following:

$$[a, b, c]$$

then the polynomial version of its features is the following (assume a max order of 2):

$$[
    \underbrace{1}_\text{product of order 0},
    \overbrace{a, b, c}^\text{products of order 1},
    \underbrace{a^2, b^2, c^2, ab, bc, ac}_\text{products of order 2}]$$

2. Uses a linear regression model, like the ones discussed above.

This way, the polynomial regression strategy assumes a linear relationship between the features (as before), **their powers and their mixed products -up to a certain order-** and the independent variable.

We will implement polynomial regressors of order 2, based on our conclusions from the EDA notebook. More specifically, we will use 2 polynomial regressors, one which will use simple linear regression and one which uses linear regression with $l1$ and $l2$ regularization, i.e. the Elastic Net regression.

Given the graphs we saw in the EDA notebook, we expect the majority of the "relatively slow" kernels to be better fitted by the polynomial regressors we will implement.

#### 2.3.1 Linear based

In [4]:
PolynomialRegressionLinear = Pipeline(
    [
        ('polyfeatures', PolynomialFeatures(degree=2)),
        ('regression', LinearRegression())
    ]
)

#### 2.3.2 Elastic Net based

In [5]:
PolynomialRegressionElasticNet = Pipeline(
    [
        ('polyfeatures', PolynomialFeatures(degree=2)),
        ('regression', MultiTaskElasticNet())
    ]
)

## 3. Selecting a regression model per kernel

We will determine the best model for each kernel via a multi-model gridsearch and 5-fold cross validation process.

### 3.1 Hyperparameter optimization across multiple models

In [6]:
class MultiModelGridSearchCV:
    '''A helper class for multi-model gridsearch
    The original class was found in the following blog post:
    http://www.davidsbatista.net/blog/2018/02/23/model_optimization/
    and was heavily modified to support multiple scorers, amongst others

    Attributes
    ----------
    models : dict
        dict of models
    params : dict(dict)
        dict of parameter grids
    
    Methods
    -------
    fit : same as GridSearchCV, but for every model
        in the model list
    scores : returns a list of dictionaries (one for each model/gridsearch)
             which contain the following:
             - model_name : the name of the best model (estimator)
             - best_estimator : estimator with the best score
             - r2 : the best r2 score achieved, i.e. the best_estimator r2 score
    '''

    def __init__(self, models, params):
        if not set(models.keys()).issubset(set(params.keys())):
            missing_params = list(set(models.keys()) - set(params.keys()))
            raise ValueError('Some estimators are missing parameters: %s' % missing_params)
        self.models = models
        self.params = params
        self.keys = models.keys()
        self.grid_searches = {}

    def fit(self, X, y, cv=5, n_jobs=1, verbose=1, scoring=None, refit=False):
        self.scorer = scoring
        for key in self.keys:
            model = self.models[key]
            params = self.params[key]
            gs = GridSearchCV(
                model, params, cv=cv, n_jobs=n_jobs,
                verbose=verbose, scoring=scoring, refit=refit
            )
            gs.fit(X, y)
            self.grid_searches[key] = gs

        return self.grid_searches

    def scores(self, x, y):
        # return score FOR ALL MODELS
        model_scores = []
        for k, gs in self.grid_searches.items():
            curr_best_estimator = gs.best_estimator_
            model_scores.append(dict(
                model_name=k,
                best_estimator=curr_best_estimator,
                r2_score=curr_best_estimator.score(x, y)
            ))
        return model_scores

### 3.2 Final model selection

In [7]:
from oclude.utils.constants import llvm_instructions

# get the profiling files per kernel
grouped_profilings_filenames = get_grouped_all_profilings_filenames()

alpha = np.geomspace(1e-2, 1e4, 10)
l1_ratio = np.arange(0.1, 1.01, .05)
selection = ['cyclic', 'random']

# parameter grid for Elastic Net
enet_param_grid = dict(
    alpha = alpha,
    l1_ratio = l1_ratio,
    selection = selection
)

# parameter grid for Polynomial Regression based on Elastic Net
poly_l_param_grid = dict(polyfeatures__degree=[2])

# parameter grid for Polynomial Regression based on Elastic Net
poly_enet_param_grid = dict(
    polyfeatures__degree = [2],
    regression__alpha = alpha,
    regression__l1_ratio = l1_ratio,
    regression__selection = selection
)

# list of regressor models
models = [
    LinearRegression(),
    MultiTaskElasticNet(),
    PolynomialRegressionLinear,
    PolynomialRegressionElasticNet
]

# list of regressor models names
models_names = [
    'Linear Regression',
    'Elastic Net',
    'Polynomial Regression (Linear)',
    'Polynomial Regression (Elastic Net)'
]

# list of regressor models parameter grids
pgrids = [
    {},
    enet_param_grid,
    poly_l_param_grid,
    poly_enet_param_grid    
]

models = dict(zip(models_names, models))
pgrids = dict(zip(models_names, pgrids))

In [8]:
%%time

# store results in a list of dicts
model_scores = {}

# define our scorer
scorer = 'r2'

# do the following for each kernel, to save memory
n_kernels = len(grouped_profilings_filenames)
for i, (kernel_info, kproffiles) in enumerate(grouped_profilings_filenames, 1):

    print(
        f'[{i:02d}/{n_kernels}] training/testing regression models for {"/".join(kernel_info)}... ',
        end=''
    )
    
    # step 1: load the data
    prof_dict = load_profiling_dicts(kproffiles)
    
    # step 2: take the average of the data across same gsize profilings
    averaged_prof_dict = get_average_instcounts_dicts(prof_dict)

    # step 3: turn kernel data into dataframe
    kdata = pd.DataFrame(columns=['gsize'] + llvm_instructions, dtype=int)
    for i, sample in enumerate(averaged_prof_dict):
        kdata.loc[i] = [sample['gsize']] + list(sample['results']['instcounts'].values())

    ### at this point, we have a dataframe with all the averaged counts:       ###
    ### the first column is the gsizes of the profilings of this kernel        ###
    ### the rest of the columns are all the llvm instructions and their counts ###

    # step 4: x is the gsize column (input), y is all the others (target)
    x = kdata.iloc[:, :1] # shape -> (n_samples, 1)
    Y = kdata.iloc[:, 1:] # shape -> (n_samples, n_llvm_instructions)

    # step 5: split data into test and training set
    x_train, x_test, Y_train, Y_test = train_test_split(x, Y, test_size=.3)

    # step 6: select a model (grid-search and cross-validation)!
    clf = MultiModelGridSearchCV(models, pgrids)
    clf.fit(x_train, Y_train, cv=5, scoring=scorer, n_jobs=-1, verbose=0, refit=scorer)
    curr_models_stats = clf.scores(x_test, Y_test)
    model_scores[tuple(kernel_info)] = curr_models_stats

    print('done.')

[01/39] training/testing regression models for b+tree/kernel_gpu_opencl.cl/findK... done.
[02/39] training/testing regression models for b+tree/kernel_gpu_opencl_2.cl/findRangeK... done.
[03/39] training/testing regression models for backprop/backprop_kernel.cl/bpnn_adjust_weights_ocl... done.
[04/39] training/testing regression models for backprop/backprop_kernel.cl/bpnn_layerforward_ocl... done.
[05/39] training/testing regression models for bfs/Kernels.cl/BFS_1... done.
[06/39] training/testing regression models for bfs/Kernels.cl/BFS_2... done.
[07/39] training/testing regression models for dwt2d/com_dwt.cl/c_CopySrcToComponent... done.
[08/39] training/testing regression models for dwt2d/com_dwt.cl/c_CopySrcToComponents... done.
[09/39] training/testing regression models for gaussian/gaussianElim_kernels.cl/Fan1... done.
[10/39] training/testing regression models for gaussian/gaussianElim_kernels.cl/Fan2... done.
[11/39] training/testing regression models for heartwall/kernel_gpu_

/home/sotiris/projects/thesis_projects/oclude/env/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:1914: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations. Duality gap: 7.449713778903943e+36, tolerance: 3.566726342609636e+33

/home/sotiris/projects/thesis_projects/oclude/env/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:1914: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations. Duality gap: 1.2546725576721602e+37, tolerance: 3.566726342609636e+33



done.
[25/39] training/testing regression models for particlefilter/particle_double.cl/normalize_weights_kernel... done.
[26/39] training/testing regression models for particlefilter/particle_double.cl/sum_kernel... done.
[27/39] training/testing regression models for particlefilter/particle_naive.cl/particle_kernel... done.
[28/39] training/testing regression models for particlefilter/particle_single.cl/find_index_kernel... done.
[29/39] training/testing regression models for particlefilter/particle_single.cl/likelihood_kernel... 

/home/sotiris/projects/thesis_projects/oclude/env/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:1914: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations. Duality gap: 5.36817713675055e+36, tolerance: 2.0261813289159987e+33

/home/sotiris/projects/thesis_projects/oclude/env/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:1914: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations. Duality gap: 6.788341892186117e+36, tolerance: 2.0261813289159987e+33



done.
[30/39] training/testing regression models for particlefilter/particle_single.cl/normalize_weights_kernel... done.
[31/39] training/testing regression models for particlefilter/particle_single.cl/sum_kernel... done.
[32/39] training/testing regression models for pathfinder/kernels.cl/dynproc_kernel... done.
[33/39] training/testing regression models for srad/kernel_gpu_opencl.cl/compress_kernel... done.
[34/39] training/testing regression models for srad/kernel_gpu_opencl.cl/extract_kernel... done.
[35/39] training/testing regression models for srad/kernel_gpu_opencl.cl/prepare_kernel... done.
[36/39] training/testing regression models for srad/kernel_gpu_opencl.cl/reduce_kernel... done.
[37/39] training/testing regression models for srad/kernel_gpu_opencl.cl/srad2_kernel... done.
[38/39] training/testing regression models for srad/kernel_gpu_opencl.cl/srad_kernel... done.
[39/39] training/testing regression models for streamcluster/Kernels.cl/memset_kernel... done.
CPU times: us

### 3.3 Model selection results

Firstly, we will plot the mean $R^2$ scores for each one of the 4 models that we used, regardless of whether or not they were they had the best score for none, one or more kernels:

In [9]:
x = models_names
selected_model_scores = {}
y_all, y_selected = [], []
for model_name in models_names:
    y_all.append(np.mean(
        [x['r2_score'] for kinfo in model_scores.values() for x in kinfo if x['model_name'] == model_name]
    ))

fig = go.Figure(data=[go.Bar(
    x=x, y=y_all,
    text=list(map('{:.3f}'.format, y_all)),
    textposition='auto',
    marker_color='darkgreen'
)])

fig.update_layout(
    title=dict(
        text='Mean R2 score by regression model',
        font=dict(
            family='Courier New, monospace',
            size=22
        ),
        x=.5
    ),
    font=dict(
        family='Courier New, monospace',
        size=10
    )
)

fig.show()

The results confirm that our assumptions regarding the sufficiency of the linear models and the polynomial models of order no more than 2 were correct; the mean $R^2$ score is very satisfactory for all 4 models, which means that the predictions of the models we chose explain the variance of the data to a large extent.

Let us now see which models are the more "popular" ones, i.e. they have the greatest $R^2$ score for more kernels than the other models:

In [10]:
models_popularity = defaultdict(int)

for kernel, stats in model_scores.items():
    models_popularity[max(stats, key=lambda x : x['r2_score'])['model_name']] += 1

labels = models_names
values = [models_popularity[model_name] for model_name in labels]
colors = 'green', 'orange', 'red', 'blue'

fig = go.Figure(data=[go.Pie(
    labels=labels,
    values=values,
    marker=dict(colors=colors)
)])

fig.update_layout(
    title=dict(
        text='Regression models popularity',
        font=dict(
            family='Courier New, monospace',
            size=22
        ),
        x=.5
    ),
    font=dict(
        family='Courier New, monospace',
        size=12
    )
)

fig.show()

We see that the most popular model is the **polynomial regressor based on the linear regressor** (12 kernels). **Linear regression** (10 kernels) comes next, followed by **Elastic Net** (9 kernels) and **polynomial regression based on Elastic Net** (8 kernels).

Lastly, it should be quite interesting to see which regression models were preferred by the kernels *based on whether they were categorized as "relatively fast" or "relatively slow"* in the EDA notebook:

In [11]:
relatively_fast_kernels = [
    ('backprop', 'backprop_kernel.cl', 'bpnn_adjust_weights_ocl'),
    ('backprop', 'backprop_kernel.cl', 'bpnn_layerforward_ocl'),
    ('bfs', 'Kernels.cl', 'BFS_2'),
    ('dwt2d', 'com_dwt.cl', 'c_CopySrcToComponent'),
    ('dwt2d', 'com_dwt.cl', 'c_CopySrcToComponents'),
    ('gaussian', 'gaussianElim_kernels.cl', 'Fan1'),
    ('gaussian', 'gaussianElim_kernels.cl', 'Fan2'),
    ('hybridsort', 'bucketsort_kernels.cl', 'bucketsort'),
    ('hybridsort', 'mergesort.cl', 'mergeSortFirst'),
    ('myocyte', 'kernel_gpu_opencl.cl', 'kernel_gpu_opencl'),
    ('nn', 'nearestNeighbor_kernel.cl', 'NearestNeighbor'),
    ('particlefilter', 'particle_double.cl', 'find_index_kernel'),
    ('particlefilter', 'particle_double.cl', 'normalize_weights_kernel'),
    ('particlefilter', 'particle_double.cl', 'sum_kernel'),
    ('particlefilter', 'particle_naive.cl', 'particle_kernel'),
    ('particlefilter', 'particle_single.cl', 'find_index_kernel'),
    ('particlefilter', 'particle_single.cl', 'normalize_weights_kernel'),
    ('particlefilter', 'particle_single.cl', 'sum_kernel'),
    ('srad', 'kernel_gpu_opencl.cl', 'compress_kernel'),
    ('srad', 'kernel_gpu_opencl.cl', 'extract_kernel'),
    ('srad', 'kernel_gpu_opencl.cl', 'prepare_kernel'),
    ('srad', 'kernel_gpu_opencl.cl', 'reduce_kernel'),
    ('srad', 'kernel_gpu_opencl.cl', 'srad2_kernel'),
    ('srad', 'kernel_gpu_opencl.cl', 'srad_kernel'),
    ('streamcluster', 'Kernels.cl', 'memset_kernel')
]

model_scores_relatively_fast = {}
model_scores_relatively_slow = {}
for k, stats in model_scores.items():
    if k in relatively_fast_kernels:
        model_scores_relatively_fast[k] = stats
    else:
        model_scores_relatively_slow[k] = stats

models_pop_fast = defaultdict(int)
models_pop_slow = defaultdict(int)

for kernel, stats in model_scores_relatively_fast.items():
    models_pop_fast[max(stats, key=lambda x : x['r2_score'])['model_name']] += 1

for kernel, stats in model_scores_relatively_slow.items():
    models_pop_slow[max(stats, key=lambda x : x['r2_score'])['model_name']] += 1

In [12]:
labels = models_names
values_fast = [models_pop_fast[model_name] for model_name in labels]
values_slow = [models_pop_slow[model_name] for model_name in labels]
colors = 'green', 'orange', 'red', 'blue'

fig = make_subplots(
    rows=1, cols=2,
    specs=[[{'type': 'pie'}, {'type': 'pie'}]],
    subplot_titles=['...relatively fast', '...relatively slow']
)

### r2 pie chart ###

for (row, col), name, values in zip(
        [(1, 1), (1, 2)],
        ['fast kernels', 'slow kernels'],
        [values_fast, values_slow]):
    fig.add_trace(go.Pie(
        name=name,
        labels=labels,
        values=values,
        marker=dict(colors=colors)
    ), row=row, col=col)

fig.update_layout(
    title=dict(
        text='Regression models popularity among the...',
        font=dict(
            family='Courier New, monospace',
            size=22
        ),
        x=.5
    ),
    font=dict(
        family='Courier New, monospace',
        size=12
    )
)
fig.show()

As expected, the **linear models** (linear regression and Elastic Net) **were selected by the majority of the "relatively fast" kernels at a rate of 60%**, while the **polynomial models** (polynomial regression based on simple linear regression and polynomial regression based on Elastic Net) **were selected by the majority of the "relatively fast" kernels at a rate of 71.4%**.